# 第3章: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される
各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

In [3]:
import re
import gzip
import json
import requests
from collections import OrderedDict
data = []
with gzip.open('./jawiki-country.json.gz', 'r') as f:
    text = f.read().decode('utf-8').strip()
    jsons = text.strip().split("\n");
    for line in jsons:
        data.append(json.loads(line))

FileNotFoundError: [Errno 2] No such file or directory: './jawiki-country.json.gz'

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
dic = {a["title"]: a["text"] for a in data}
uk_text = dic["イギリス"]
with open('./uk_text.txt', 'w', encoding='utf-8') as fw:
    fw.write(uk_text)
print(uk_text)

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
p = r".*Category.*"
for m in re.finditer(p, uk_text, re.M):
    print(m.group(0))

## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
p = r"^\[\[Category:(.*)\]\]$"
for m in re.finditer(p, uk_text, re.M):
    print(m.group(1))

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [ ]:
p = r"^(=+)(.*)\1$"

for m in re.finditer(p, uk_text, re.M):
    print(str(len(m.group(1)) - 1) + " " + m.group(2))

## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．



In [ ]:
p = r"\[\[File:(.*?)(\|.*)?\]\]"
for m in re.finditer(p, uk_text, re.M):
    print(m.group(1))

In [ ]:
## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [2]:
p = r"\{\{基礎情報.*?\n(.*?)\n\}\}"
m = re.search(p, uk_text, re.M|re.S)
if not m:
    exit()

base_text = m.group(1)
base_dic = OrderedDict()
p = r"^\|([^|]*?) = (.*?)(?=\n\|)"
for m2 in re.finditer(p, base_text, re.M|re.S):
    key, value = m2.groups()
    base_dic[key] = value

for key, value in base_dic.items():
    print(" " + key + "\n" + value + "\n")

NameError: name 'uk_text' is not defined

In [ ]:
## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
base_dic2 = OrderedDict()
for key, value in base_dic.items():
    base_dic2[key] = re.sub(r"'''(.*)'''", r"\1", value, re.M|re.S)

for key, value in base_dic2.items():
    print(" " + key + "\n"+ value + "\n")

In [ ]:
## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
base_dic3 = OrderedDict()
for key, value in base_dic2.items():
    base_dic3[key] = re.sub(r"\[\[(?:.*\|)?(.*?)\]\]", r"\1", value, re.M|re.S)

for key, value in base_dic3.items():
    print(" " + key + "\n"+ value + "\n")

In [ ]:
## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
base_dic4 = OrderedDict()
for key, value in base_dic3.items():
    value = re.sub(r"\{\{(?:.*\|)?(.*?)\}\}", r"\1", value, re.M|re.S)
    value = re.sub(r"(?m)^\*+", "", value)
    value = re.sub(r"<.* ?/?>", "", value, re.M|re.S)
    base_dic4[key] = value

for key, value in base_dic4.items():
    print(" " + key + "\n"+ value + "\n")

In [ ]:
## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [ ]:
import requests
r = requests.get('https://commons.wikimedia.org/w/api.php?action=query&titles=File:Flag%20of%20the%20United%20Kingdom.svg&prop=imageinfo&&iiprop=url&iiurlwidth=220&format=json')
res = r.json()

In [ ]:
url = res['query']['pages']['347935']['imageinfo'][0]['url']
url